In [6]:
import csv
import sys
import numpy as np
import sklearn as sk

In [7]:
print(sys.version)

3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]


In [8]:
male_data = np.genfromtxt('Data/MALE.csv', delimiter=',', skip_header=1)
female_data = np.genfromtxt('Data/FEMALE.csv', delimiter=',', skip_header=1)
mixed_data = np.genfromtxt('Data/MIXED.csv', delimiter=',', skip_header=1)

male_x = male_data[:,:-1]
male_y = male_data[:,-1:]
female_x = female_data[:,:-1]
female_y = female_data[:,-1:]
mixed_x = mixed_data[:,:-1]
mixed_y = mixed_data[:,-1:]

In [12]:
from sklearn.model_selection import train_test_split

male_x_train, male_x_test, male_y_train, male_y_test = \
train_test_split(male_x, male_y, test_size=0.1, random_state=1)

male_x_train, male_x_validation, male_y_train, male_y_validation = \
train_test_split(male_x_train, male_y_train, test_size=0.2222, random_state=1) 

female_x_train, female_x_test, female_y_train, female_y_test = \
train_test_split(female_x, female_y, test_size=0.1, random_state=1)

female_x_train, female_x_validation, female_y_train, female_y_validation = \
train_test_split(female_x_train, female_y_train, test_size=0.2222, random_state=1) 

mixed_x_train, mixed_x_test, mixed_y_train, mixed_y_test = \
train_test_split(mixed_x, mixed_y, test_size=0.1, random_state=1)

mixed_x_train, mixed_x_validation, mixed_y_train, mixed_y_validation = \
train_test_split(mixed_x_train, mixed_y_train, test_size=0.2222, random_state=1) 